In [129]:
import requests
import polyline
import json
import pandas as pd
from shapely.geometry import Point
import folium
import geopandas as gpd
import dotenv
import os

In [130]:
dotenv.load_dotenv()
ORS_API_KEY=os.getenv('ORS_API_KEY')
AZURE_API_KEY=os.getenv('AZURE_API_KEY')

In [145]:
def obtener_altitudes(coordinates, api_key):
    url = "https://api.openrouteservice.org/elevation/line"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }

    # ORS espera formato lat, lon para ENCODED_POLYLINE
    #geometry_latlng = [(round(lat, 5), round(lng, 5)) for lat, lng in coordinates]
    geometry_latlng = [(lat, lng) for lat, lng in coordinates]

    payload = {
        "format_in": "encodedpolyline",
        "format_out": "geojson",
        "geometry": polyline.encode(geometry_latlng)
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        raise Exception(f"Error elevación: {response.text}")
    data = response.json()
    return data['geometry']['coordinates']

In [182]:
def get_route(coords,api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    payload = {
        "coordinates": coords,
        "instructions": True,
        #"elevation":True
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error elevación: {response.text}")
    
    data = response.json()
    return data

In [136]:
def get_opt_route(coords,api_key):
    features = []
    for idx, (lon, lat) in enumerate(coords):
        point_type = "waypoint" if idx in (0, len(coords) - 1) else "viaWaypoint"
        features.append({
            "type": "Feature",
            "geometry": {
                "coordinates": [lon, lat],
                "type": "Point"
            },
            "properties": {
                "pointIndex": idx,
                "pointType": point_type
            }
        })

    body = {
        "type": "FeatureCollection",
        "features": features,
        "optimizeRoute": "fastestWithTraffic",
        "routeOutputOptions": ["itinerary","routePath"],
        "maxRouteCount": 1,
        "travelMode": "driving",
        "departureAt": "2025-08-14T08:00:00-05:00"
    }

    url = "https://atlas.microsoft.com/route/directions"
    params = {"api-version": "2025-01-01"}

    response = requests.post(
        url,
        params=params,
        headers={
            "Content-Type": "application/json; charset=UTF-8",
            "subscription-key": api_key
        },
        json=body
    )
    response.raise_for_status()
    return response.json()

In [188]:
# Asignar tiempos reales y etiquetas desde steps
def get_vel(steps,elevation_data):
    route = []
    total_time = 0
    total_distance = 0

    for step in steps:
        start_idx, end_idx = step["way_points"]
        step_duration = step["duration"]  # segundos
        step_distance = step["distance"]  # metros
        num_points = end_idx - start_idx

        # Evitar división por cero
        if num_points == 0:
            continue

        # Duración por punto
        duration_per_point = step_duration / num_points
        distance_per_point = step_distance / num_points

        for i in range(num_points):
            idx = start_idx + i
            lng, lat, alt = elevation_data[idx]

            if len(route) == 0:
                prev_alt = alt
            else:
                prev_alt = route[-1]["altitude"]

            delta_alt = alt - prev_alt
            slope = (delta_alt / distance_per_point * 100) if distance_per_point != 0 else 0

            route.append({
                "latitude": (lat),
                "longitude": (lng),
                "altitude": round(alt, 2),
                "distance_mt": round(total_distance, 2),
                "distance_km": round(total_distance / 1000, 2),
                "time_sec": round(total_time, 2),
                "slope_pct": round(slope, 2),
                "speed_ms": round((distance_per_point / duration_per_point), 2),
                "speed_kmh": round((distance_per_point / duration_per_point) * 3.6, 2),
            })
            total_time += duration_per_point
            total_distance += distance_per_point
    return route

In [2]:
df_med=pd.read_json("results/med.json")
df_med=df_med.iloc[[1]]

In [3]:
coords=[
    [df_med.iloc[0]['o_long'],df_med.iloc[0]['o_lat']],
    [df_med.iloc[0]['d_long'],df_med.iloc[0]['d_lat']]
]

In [137]:
response_opt=get_opt_route(coords,AZURE_API_KEY)

In [ ]:
data=get_route(coords, ORS_API_KEY)
geometry_encoded = data["routes"][0]["geometry"]
geometry = polyline.decode(geometry_encoded)

In [ ]:
elevation_data = obtener_altitudes(geometry, ORS_API_KEY)
steps = data["routes"][0]["segments"][0]["steps"]

In [189]:
route=get_vel(steps,elevation_data)

In [ ]:
with open("vel_route.json", "w", encoding="utf-8") as f:
        json.dump(route, f, ensure_ascii=False, indent=2)

with open("ors_route.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

with open("azure_route.json", "w", encoding="utf-8") as f:
        json.dump(response_opt, f, ensure_ascii=False, indent=2)

In [120]:
'''
for index, row in df_med.iterrows():

    if(row['route'] is not None):

        continue

    #row['route']=get_route()
'''

"\nfor index, row in df_med.iterrows():\n\n    if(row['route'] is not None):\n\n        continue\n\n    #row['route']=get_route()\n"

In [55]:
with open("ors_route.json") as f:
    data_ors = json.load(f)

with open("azure_route.json") as f:
    data_azure = json.load(f)

In [101]:
geometry_azure=data_azure['features'][-1]['geometry']['coordinates'][0]
geometry_azure=[[lat,lon] for lon,lat in geometry_azure]

geometry_ors=data_ors['routes'][0]['geometry']
geometry_ors=polyline.decode(geometry_ors)

In [118]:
# [Lat,lon] format
def generate_map(coords):
    m=folium.Map(location=coords[0], zoom_start=12)
    folium.Marker(coords[0]).add_to(m)
    folium.Marker(coords[-1]).add_to(m)
    folium.PolyLine(
        coords,
        color="blue",
        weight=5,
        opacity=0.8
    ).add_to(m)
    return m

In [119]:
m=generate_map(geometry_azure)
m.save("azure_map.html")

m=generate_map(geometry_ors)
m.save("ors_map.html")